In [22]:
import pandas as pd

df = pd.read_csv('../final.csv')
df.head()

,회사이름,공고이름,url,경력,지역,직무
0,㈜인라이플,[주4.5일근무] 데이터 분석 및 기획자 채용,https://www.jobkorea.co.kr/Recruit/GI_Read/457...,경력2년↑,서울,데이터 분석
1,패스트파이브㈜,데이터 분석 담당자,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력5년↑,서울,데이터 분석
2,주식회사 빗썸,[빗썸] 비즈니스 데이터 분석,https://www.jobkorea.co.kr/Recruit/GI_Read/459...,경력7년↑,서울,데이터 분석
3,㈜컬리,[컬리] 온사이트 프로모션 데이터 분석,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력,서울,데이터 분석
4,"㈜숲(SOOP CO., LTD.)",[soop] 데이터 분석 및 지표 설계,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력3년↑,경기,데이터 분석


In [23]:
# 'SearchText' 열을 구조화된 텍스트로 생성
df['SearchText'] = (
    "회사이름: " + df['회사이름'] + ", "
    "공고이름: " + df['공고이름'] + ", "
    "URL: " + df['url'] + ", "
    "경력: " + df['경력'] + ", "
    "지역: " + df['지역'] + ", "
    "직무: " + df['직무']
)

In [24]:
df['SearchText'].isnull().sum()

0

In [25]:
# print(df[df['SearchText'].isnull()])

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import numpy as np

In [27]:
# 3. 벡터화 (TF-IDF 사용 예제)
vectorizer = TfidfVectorizer(max_features=512)  # 차원 설정
X = vectorizer.fit_transform(df['SearchText']).toarray()

In [28]:
X.shape

(2365, 512)

In [29]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.05612876],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.05472454],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.04379234],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07958746],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11460265],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.09602668]])

In [30]:
# FAISS 인덱스 생성
dimension = X.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(X, dtype=np.float32))

In [31]:
print(index.reconstruct_n(0, 5))

[[0.         0.         0.         ... 0.         0.         0.05612876]
 [0.         0.         0.         ... 0.         0.         0.05472454]
 [0.         0.         0.         ... 0.         0.         0.04379234]
 [0.         0.         0.         ... 0.         0.         0.04554772]
 [0.         0.         0.         ... 0.         0.         0.04416253]]


In [32]:
print(index.ntotal)

2365


In [33]:
# # 인덱스 저장
# faiss.write_index(index, 'job_postings.index')

In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
import os

In [54]:
# import langchain
# import faiss

# print("Langchain version:", langchain.__version__)
# print("Faiss version:", faiss.__version__)


Langchain version: 0.2.17
Faiss version: 1.9.0


In [37]:
# 문서 데이터 생성
docs = [{"id": str(i), "text": row["SearchText"]} for i, row in df.iterrows()]

In [42]:
docs

[{'id': '0',
  'text': '회사이름: ㈜인라이플, 공고이름: [주4.5일근무] 데이터 분석 및 기획자 채용, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45711948?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=1, 경력: 경력2년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '1',
  'text': '회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46017897?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=2, 경력: 경력5년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '2',
  'text': '회사이름: 주식회사 빗썸, 공고이름: [빗썸] 비즈니스 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45906918?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=3, 경력: 경력7년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '3',
  'text': '회사이름: ㈜컬리, 공고이름: [컬리] 온사이트 프로모션 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46002356?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=4, 경력: 경력, 지역: 서울, 직무: 데이터 분석'},
 {'id': '4',
  'text': '회사이름: ㈜숲(SOOP CO.

In [35]:
openai_api_key = input("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [63]:
embeddings = OpenAIEmbeddings()

In [64]:
from langchain.docstore.document import Document

# dict를 Document 객체로 변환
documents = [Document(page_content=doc["text"], metadata={"id": doc["id"]}) for doc in docs]
from langchain.vectorstores import FAISS

# FAISS 인덱스 생성
vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)


In [81]:
vectorstore

In [71]:
llm = ChatOpenAI(model = 'gpt-4o', temperature=0.1)

In [74]:
question = '서울에 데이터 분석가의 채용공고를 알려줘'

In [84]:
# FAISS에서 데이터 검색
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # 상위 5개 검색

# 검색된 문서 추출
retrieved_docs = retriever.get_relevant_documents(question)

# 검색 결과를 context로 변환
context = "\n".join([doc.page_content for doc in retrieved_docs])

# Prompt 템플릿 정의
prompt = PromptTemplate.from_template(
    """
    너는 주어진 구인공고 질문에 대답하는 AI야. 다음 검색된 context를 사용해서 question에 대답해줘.
    구인공고 정보를 제공해주는데, 회사이름 : , 공고이름: , 지역: ,url: , 형식으로 대답해줘. 5개를 구인공고 정보를 제공해 주는데, 
    제목에 직무가 속한 데이터를 먼저 소개해주고 없다면, 랜덤으로 제공해줘.
    답을 모르면, '알 수 없습니다'라고 대답해

    # Context : {context}
    # Question : {question}
    # Answer :
    """
)

# LLM 설정
llm = ChatOpenAI(model='gpt-4', temperature=0.1)

# 체인 구성
chain = LLMChain(llm=llm, prompt=prompt)

# 질문에 대해 답변 생성
result = chain.run({"context": context, "question": question})

print(result)

1. 회사이름: ㈜에이브랩스, 공고이름: 데이터 분석 (data scientist) 채용, 지역: 서울, url: https://www.jobkorea.co.kr/Recruit/GI_Read/46057474?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=41
2. 회사이름: 씨앤토트㈜, 공고이름: 데이터 분석 및 시각화 (경력), 지역: 서울, url: https://www.jobkorea.co.kr/Recruit/GI_Read/46025939?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=15
3. 회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, 지역: 서울, url: https://www.jobkorea.co.kr/Recruit/GI_Read/46017897?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=2
4. 회사이름: ㈜해양정보기술, 공고이름: 해양빅데이터 분석 및 해양수치모델링 부문 정규직 채용, 지역: 서울, url: https://www.jobkorea.co.kr/Recruit/GI_Read/45913695?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=47
5. 회사이름: 씨앤토트㈜, 공고이름: 빅데이터 분석 시스템 모델러, 데이터 아키텍트 (da) 모집 (경력), 지역: 서울, url: https://www.jobkorea.co.kr/Recruit/GI_Read/46003102?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno